# Generator

### reference<br>
-------------------------------------------------
https://docs.python.org/3/library/itertools.html#module-itertools

### generator를 배우기 앞서
--------------------------------------------------
1. procedural<br>
2. object-oriented<br>
3. functional<br>
위 세 가지의 프로그래밍 방법이 있다.

functional programming은 문제를 여러가지의 함수로 분해한다.<br>
그리고 위 함수들은 어떠한 output에 영향을 주는 internal state없이 input이 주어지면<br>
같은 output을 생성하도록 구현된다.<br>

python은 위 세 가지의 방법 모두를 지원하며 꼭 위 세가지 중 하나를 선택할 필요는 없다.<br>
functional programming을 위해서는 I/O도, assignment도 하지 않아야하는 극단적인<br>
선택을 해야하는데 파이썬은 그런 방법을 선택하지 않고<br>
__functional-approaching-interface__를 제공한다.<br>

Object-oriented programming이 작은 capule 안에서 method를 호출하며<br>
internal state를 바꿔가 프로그램이 진행된다면

Functional programming은 internal-state의 변화를 최소화하며<br>
오직 함수 사이의 data flow 만으로 문제를 해결하는 방법이다.

Python에서는 위 두가지 방법 모두를 사용한다.

iterator<br>
---------------------------------------------------------

- 먼저 functional programming의 foundation인 iterator를 살펴보자

1. iterator는 stream of data를 나타내는 object이다.<br>
2. iterator는 반드시 \_\_next__ magic method가 있어야하며 arguments는 없고<br>
   항상 다음 elements를 return한다.<br>
   만약 다음 elem이 없으면 StopIteration을 raise한다.<br>
3. 꼭 finite하게 구현할 필요는 없으며 infinite하게 구현하는 것이 훨씬 낫다.

- built-int function인 \_\_iter__는 arbitrary object을 args로 받는다.<br>
  만약 object이 iteration을 지원하지 않는다면 _TypeError_를 raise한다.
- iterator가 존재하는 object을 __iterable__이라고 한다.

In [2]:
L = [1, 2, 3]
it = iter(L)
print(next(it))
print(next(it))
print(next(it))
print(next(it))

1
2
3


StopIteration: 

python에서는 여러 경우의 iterator를 요구하는데<br>
대표적인 예가 for문이다.
```python
    for X in Y
```
에서 Y는 iterator를 꼭 가져야한다.<br>
아래 두 코드는 동일하다.<br>
```python
    for i in iter(obj):
        print(i)

    for i in obj:
        print(i)
```

max(), min() built-in function을 사용할 수 있으며
_in_, _not in_ 도 iterator가 정의되어야한다.

- python에서 지원하는 대부분의 sequential은 iterator를 지원한다.
- list, tuple, dictionary, string도 내부적으로 iteration을 생성한다.
- 특별히 dictionary 에서는 _keys()_, _values()_ 라는 method로<br>
  다른 iter를 생성할 수 있다.

- iter가 output을 생성하는 방법에는 보통 두 가지가 있다.
1. 모든 elements에 대하여 각각 operate<br>
2. 특정 조건을 만족하는 subset에 대하여 operate<br>

- 짧게 "listcomp", "genexps"이라고 하는
list comprehension, generator expressions 는 functional programming language인<br>
Haskell에서 빌려온 표현이다.

In [5]:
line_list = ['     line1  \n', '    line2  \n', 'line3    \n', '']

# generator expression
stripped_iter = (line.strip() for line in line_list)
# list comprehension
stripped_list = [line.strip() for line in line_list]

print(stripped_iter)
print(stripped_list)

<generator object <genexpr> at 0x7ff7494b0f20>
['line1', 'line2', 'line3', '']


특정 조건에 맞는 element만 뽑고 싶다면 아래와 같이 작성할 수도 있다.<br>

In [6]:
stripped_condition_list = [
    line.strip() for line in line_list if line != ''
]
print(stripped_condition_list)

['line1', 'line2', 'line3']


```python
    (expression for exp1 in seq1
                if condition1
                for exp2 in seq2
                if condition2
                for expN in seqN
                if conditionN )
```
위 genexps는 아래의 코드 진행 순서와 일치한다.<br>
```python
    for exp1 in seq1:
        if not condition1:
            continue
        for exp2 in seq2:
            if not condition2:
                continue
            ...
            for expN in seqN:
                if not conditionN:
                    continue
```

- python이 모호하게 해석하지 않게 만약 generator가 tuple을 생성한다면<br>
  () 로 감싸줘야한다.
```python
    [(x, y) for x in seq1 for y in seq2]
```

Generator의 기본 아이디어는 다음과 같다.<br>
기존의 C, python이 함수를 호출하는 방식은 각 함수만의 private namespace를 생성하고<br>
내부 계산을 수행한 뒤 결과를 return하고 생성한 namespace를 날려버린다.<br>
그리고 다음에 다시 함수가 호출되면 새로 private namespace를 생성하고 같은 작업을 반복한다.<br>

그러나 만약 함수가 그 private namespace를 날리지 않고 잠시 멈췄다가 다시 실행할 수 있다면..?<br>
이것이 generator 함수의 기본 아이디어이다.<br>

In [8]:
def generate_ints(N):
    for i in range(N):
        yield i

- __yeild__ keyword가 있는 모든 함수는 generator 함수이다.

보통 아래와 같이 사용한다.

In [9]:
gen = generate_ints(3)
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

0
1
2


StopIteration: 

# Passing values into generator

value는 send(value) method를 call함으로써 generator안에 전달될 수 있다.<br>
send(value)는 generator code를 실행하고 value를 generator 안에서 return한다.

아래 예제를 살펴보자

In [12]:
def counter(N):
    i = 0
    while i <= N:
        val = (yield i)
        if val is not None:
            i = val
        else:
            i += 1

it = counter(10)
print(next(it))
print(next(it))
print(next(it))
it.send(5)
print(next(it))
print(next(it))
print(next(it))

0
1
2
6
7
8


위 generator code 안에서 yield는 일반적인 \_\_next__ method는
None을 return한다.

# Chapter7 제네레이터 사용하기
-----------------------------------------

### 목표
1. 프로그램의 성능을 향상시키는 제네레이터 만들기<br>
2. 이터레이터가 파이썬에 어떻게 환전히 통합되었는지 확인<br>
3. 이터레이션 문제를 이상적으로 해결하는 방법<br>
4. 제네레이터가 어떻게 코루틴과 비동기 프로그래밍의 기반이 되는 역할을 하는지 확인<br>
5. 코루틴을 지원하기 위한 yield from, await, async, def와 같은 문법의 세부 기능 확인<br>
--------------------------------------------

아래 코드를 살펴보자

In [ ]:
class PurchaseStatus:
    def __init__(self, purchase):
        self.purchase = iter(purchase)
        self.min_price: float = None
        self.max_price: float = None
        self._total_purchase_price: float = 0.
        self._total_purchase = 0
        self._initialize()

    def _initialize(self):
        try:
            first_value = next(self.purchase)
        except StopIteration:
            raise ValueError("No values provided")

        self.min_price = self.max_price = first_value
        self._update_avg(first_value)

    def process(self):
        for purchase_value in self.purchase:
            self._update_min(purchase_value)
            self._update_max(purchase_value)
            self._update_avg(purchase_value)
        return self

    def _update_min(self, new_value: float):
        if new_value < self.min_price:
            self.min_price = new_value

    def _update_max(self, new_value: float):
        if new_value > self.max_price:
            self.max_price = new_value

    @property
    def avg_price(self):
        return self._total_purchase_price / self._total_purchase

    def _updaste_avg(self, new_value: float):
        self._total_purchase_price += new_value
        self._total_purchase += 1

    def __str__(self):
        return (
            f'{self.__class__.__name__}({self.min_price}, ',
            f'{self.max_price}, {self.avg_price})'
        )

    # 위 클래스는 모든 구매 정보를 받아서 필요한 계산을 한다.<br>
    # 이제 이 모든 정보를 로드해서 어딘가에 담아서 반환해주는 함수를 만들어보자.<br>
def _load_purchases(filename):
    purchases = []
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(',')
            purchases.append(float(price_raw))

    return purchases

위 코드는 정상 작동하지만 파일이 상당히 많으면 시간이 너무 오래걸리고<br>
메모리에 모두 올리지 못할 만큼 클 수도 있다.<br>

앞서 작성한 코드를 살펴보면 한 번에 하나의 데이터만을 사용하는 것을 알 수 있다.<br>
이를 이용해 한번에 하나씩의 데이터만을 가져오는 것이다.<br>

이때 제네레이터를 사용한다.

In [ ]:
def load_purchases(filename):
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(',')
            yield float(price_raw)

주목할 점은 코드는 거의 동일하다는 점이다.

class에서 \_\_iter__(), \_\_next__() 메서드를 구현해야 이터레이터로 사용가능하다.

In [2]:
class SequenceOfNumbers:
    def __init__(self, start=0):
        self.current = start

    def __next__(self):
        current = self.current
        self.current += 1
        return current

    def __iter__(self):
        return self

In [3]:
list(zip(SequenceOfNumbers(), 'abcdefg'))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f'), (6, 'g')]

### Itertools
---------------------
```python
    from itertools import islice
    purchases = islice(filter(lambda p: p > 1000.0, purchases), 10)
    stats = PurchaseStatus(purchases).process()
```
위와 같이 표현할 수 있다.<br>

### 중첩 루프
-----------------------
```python
    def search_nested_bad(array, desired_value:
        coords = None
        for i, cell in enumerate(array):
            for j, cell in enumerate(array):
                if cell == desired_value:
                    coords = (i, j)
                    break
            if coords is not None:
                break
        if coords is None:
            raise ValueError(f'{desired_value} not found')
        logger.info('[%i, %i]에서 값 %r 찾음', *coords, desired_value)
        return coords
```
위와 같이 지저분한 코드를 작성해야 이중 루프에서 탈출 할 수 있다.
그러나 제네레이터를 쓰면 보다 깔끔한 코드를 작성할 수 있다.

```python
    def _iterate_array2d(array2d):
        for i, row in enumerate(array2d):
            for j, cell in enumerate(row):
                yield (i, j), cell

    def search_nested(array, desired_value):
        try:
            coords = next(
                coords
                for (coords, cell) in _iterate_array2d()
                if cell == desired_value
            )
        except StopIteration:
            raise ValueError(f'{desired_value} not found')

        logger.info('[%i, %i]에서 값 %r 찾음', *coords, desired_value)
        return coords
```
훌륭하다..

### 파이썬의 이터레이터 패턴
----------------------------
iteratorsms \_\_iter__, \_\_next__ 매직 메서드를 구현한 object이다.<br>

| concept | magic method | 내용 |
|----------|-------------|------------|
| Iterable | \_\_iter__ | 이터레이터와 함께 반복 로직을 만든다. 이것을 구현한 객체는 for in 구문에서 사용가능 |
| Iterator | \_\_next__ | 한 번에 하나씩 값을 생산하는 로직을 정의한다. 더 이상 생산할 값이 없을 경우 StopIteration을 raise |

In [7]:
class SequenceIterator:
    def __init__(self, start=0, step=1):
        self.current = start
        self.step = step

    def __next__(self):
        value = self.current
        self.current += self.step
        return value

In [9]:
si = SequenceIterator(1, 2)
print(next(si))
print(next(si))
print(next(si))
print(next(si))

for _ in si: pass

1
3
5
7


TypeError: 'SequenceIterator' object is not iterable

위 예제는 iterator는 정의하였지만 iteralbe하지 않으므로 \_\_next__는 호출가능하지만<br>
for in 구문은 사용할 수 없다.<br>

\_\_iter__가 없더라도 for in 구문을 사용할 수 있는데
\_\_len__ 과 \_\_getitem__ method가 정의되어있다면 사용가능하다.

In [10]:
import logging
logger = logging.getLogger('logger')

class MappedRange:
    """특정 숫자 범위에 대해 맵으로 변환"""
    def __init__(self, transformation, start, end):
        self._transformation = transformation
        self._wrapped = range(start, end)

    def __getitem__(self, index):
        value = self._wrapped.__getitem__(index)
        result = self._transformation(value)
        logger.info(f'Index {index}: {result}')
        return result

    def __len__(self):
        return len(self._wrapped)

In [11]:
mr = MappedRange(abs, -10, 5)
mr[0]

10

In [12]:
mr[-1]

4

In [13]:
list(mr)

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4]

이러한 구현은 대비책임을 항상 기억해야한다.<br>
이 방법보다는 \_\_iter__ 를 구현하는게 더 바람직하다.

hello

In [3]:
def sequence(name, start, end):
    print(f'{name} 제네레이터에서 {start}에서 시작')
    yield from range(start, end)
    print(f'{name} 제네레이터에서 {end}에서 종료')
    return end

def main():
    step1 = yield from sequence('first', 0, 5)
    step2 = yield from sequence('seco nd', step1, 10)
    return step1 + step2

In [2]:
class 

NameError: name 'main' is not defined

2